In [24]:
import yfinance as yf
import requests
import pandas as pd
import datetime as dt
import time
from datetime import timedelta
from prod.model import *
from functools import reduce
from yahoo_fin import stock_info as si
from yahoo_fin.stock_info import get_data
import numpy as np
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets

In [25]:
initial_amount=5000000/777.18

# ponderacion del portafolio
ponderacion={'ETHO':4.06/100,
'ESGV':4.12/100,
'VT':12.16/100,
'XLK':15.16/100,
'VTI':19.61/100,
'IVV':11.42/100,
'QQQ':15.30/100,
'ESGU':4.79/100,
'SMH':6.72/100,
'ARKW':2.30/100,
'ARKK':2.13/100
}
#'OTROS':2.23/100

ticker("esgv").getLatest()

08/13/2020
Creating ticker ...
esgv was created, date:08/13/2020


,date,open,high,low,last_price,change,price_change
0,2020-08-13,61.84,61.985001,61.529999,61.709999,-0.210222,-0.130001


In [26]:
# portafolio en el que se invierte
tickers=["ESGV","ETHO","VT","XLK","VTI","IVV","QQQ","SMH","ARKW","ARKK","ESGU"]

dataframes=[]
for i in tickers:
       
    # Generamos el diccionario pada cada ticker
    string="{i}=ticker('{i}')".format(i=i)
    exec(string)
    
    string2="df_{i}={i}.getWeeks(12)".format(i=i)
    exec(string2)
    
    string3="df_{i}['ticker']='{i}'".format(i=i)
    exec(string3)
    
    string4="dataframes.append(df_{i})".format(i=i)
    exec(string4)

08/13/2020
Creating ticker ...
ESGV was created, date:08/13/2020
08/13/2020
Creating ticker ...
ETHO was created, date:08/13/2020
08/13/2020
Creating ticker ...
VT was created, date:08/13/2020
08/13/2020
Creating ticker ...
XLK was created, date:08/13/2020
08/13/2020
Creating ticker ...
VTI was created, date:08/13/2020
08/13/2020
Creating ticker ...
IVV was created, date:08/13/2020
08/13/2020
Creating ticker ...
QQQ was created, date:08/13/2020
08/13/2020
Creating ticker ...
SMH was created, date:08/13/2020
08/13/2020
Creating ticker ...
ARKW was created, date:08/13/2020
08/13/2020
Creating ticker ...
ARKK was created, date:08/13/2020
08/13/2020
Creating ticker ...
ESGU was created, date:08/13/2020


In [34]:
# Concatenamos todos los dataframes en solo uno
day_display=reduce(lambda x, y: pd.concat([x, y]), dataframes)

,date,open,high,low,close,adjclose,volume,ticker,pond,initial_invested_usd,real_invested,not_invested,change,invested,acum,result


In [28]:


print("Getting results...")

# agregamos una columna de ponderacion de cada instrumento
day_display["pond"]=day_display["ticker"].apply(lambda x: ponderacion.get(x))

# Extraemos los montos que no tuvieron pnderacion
not_considered_per=1-sum(ponderacion.values())

# Guardamos lo que no se invirtio.. no se considera en la estimacion
rest=initial_amount*not_considered_per
invested=initial_amount-rest

# Agregamos columnas de cantidad inicial invertida y no invertida independiente del fondo
day_display["initial_invested_usd"]=initial_amount
day_display["real_invested"]=invested
day_display["not_invested"]=rest
day_display["change"]=((day_display["close"]/day_display["open"])-1)

# Calculamos lo que se invirtio en cada instrumento
day_display["invested"]=day_display["pond"].apply(lambda x: x*initial_amount)

dataframes=[]

for i in tickers:
    string1="df_{i}=day_display[day_display['ticker']=='{i}']".format(i=i)
    string2="acum=df_{i}['change'].cumsum()".format(i=i)
    string3="df_{i}['acum']=acum".format(i=i)
    string4="df_{i}['result']=(1+df_{i}['acum'])*df_{i}['invested']".format(i=i)
    string5="dataframes.append(df_{i})".format(i=i)
    
    strings=[string1,string2,string3,string4,string5]
    
    for i in strings:
        exec(i)
        
day_display=reduce(lambda x, y: pd.concat([x, y]), dataframes)

print("Saving...")

save_gsheets(1,day_display)

print("Process completed")

Getting results...
Saving...


/Users/cristianvildosola/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Process completed


In [29]:
day_display

,date,open,high,low,close,adjclose,volume,ticker,pond,initial_invested_usd,real_invested,not_invested,change,invested,acum,result
0,2020-05-21,53.689999,53.689999,53.000000,53.230000,53.037659,211400.0,ESGV,0.0412,6433.516045,6290.048637,143.467408,-0.008568,265.060861,-0.008568,262.789903
1,2020-05-22,53.080002,53.419998,52.990002,53.410000,53.217007,82200.0,ESGV,0.0412,6433.516045,6290.048637,143.467408,0.006217,265.060861,-0.002351,264.437784
2,2020-05-26,54.700001,54.790001,53.930000,53.950001,53.755058,150800.0,ESGV,0.0412,6433.516045,6290.048637,143.467408,-0.013711,265.060861,-0.016062,260.803495
3,2020-05-27,54.630001,54.750000,53.410000,54.750000,54.552166,421400.0,ESGV,0.0412,6433.516045,6290.048637,143.467408,0.002197,265.060861,-0.013865,261.385721
4,2020-05-28,55.040001,55.330002,54.490002,54.570000,54.372814,220400.0,ESGV,0.0412,6433.516045,6290.048637,143.467408,-0.008539,265.060861,-0.022405,259.122296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,2020-08-06,75.370003,75.940002,75.209999,75.930000,75.930000,267800.0,ESGU,0.0479,6433.516045,6290.048637,143.467408,0.007430,308.165419,0.001863,308.739567
54,2020-08-07,75.589996,75.976997,75.379997,75.949997,75.949997,416500.0,ESGU,0.0479,6433.516045,6290.048637,143.467408,0.004763,308.165419,0.006626,310.207218
55,2020-08-10,76.110001,76.144997,75.500000,76.050003,76.050003,373200.0,ESGU,0.0479,6433.516045,6290.048637,143.467408,-0.000788,308.165419,0.005837,309.964291
56,2020-08-11,76.339996,76.518997,75.289001,75.400002,75.400002,346300.0,ESGU,0.0479,6433.516045,6290.048637,143.467408,-0.012313,308.165419,-0.006476,306.169768


In [14]:
results=day_display[["date","result","invested"]].groupby(by=["date"]).sum()

In [19]:
results=results.reset_index()
results["change"]=((results['result']/results['invested'])-1)*100

In [22]:
results=results[["date","result","invested","change"]]

In [35]:
results

,date,result,invested,change
0,2020-05-21,6238.895907,6290.048637,-0.813233
1,2020-05-22,6258.572816,6290.048637,-0.500407
2,2020-05-26,6173.358757,6290.048637,-1.855151
3,2020-05-27,6202.137053,6290.048637,-1.397630
4,2020-05-28,6172.675854,6290.048637,-1.866008
5,2020-05-29,6223.295675,6290.048637,-1.061247
6,2020-06-01,6266.265253,6290.048637,-0.378111
7,2020-06-02,6301.895033,6290.048637,0.188336
8,2020-06-03,6334.312934,6290.048637,0.703719
9,2020-06-04,6323.575161,6290.048637,0.533009
